In [ ]:
# load a few useful libs
import tarfile
import gc
import email
import json
import re
import io
import os
import glob
import pandas

In [ ]:
message_save_dir = "./datasets/processed/messages/"
metadata_save_dir = "./datasets/processed/metadata/"

# Warning: Next cell may take too much time to be run!

In [ ]:
# let's iterate over the dataset file without opening it up on the file system

counter = 0
with tarfile.open("./datasets/sample_enron_mail_20150507.tar.gz") as dataset_file:
    for file in dataset_file.getmembers():
        try:
            file_content = dataset_file.extractfile(file).read()
            parsed_message = email.message_from_string(file_content.decode("utf-8"))
            metadata = dict(parsed_message.items())
            print(metadata['To'])
            metadata['To'] = re.sub(r'[\n\r\t ]+', '', metadata['To']).split(",")
            print(metadata['To'])
            message = parsed_message.get_payload()
            
            # write out the message
            with io.open(os.path.join(message_save_dir, metadata['Message-ID']+".txt"), "w") as message_datafile:
                message_datafile.write(message)
            
             # write out the message metadata(as json)
            with io.open(os.path.join(metadata_save_dir, metadata['Message-ID']+".json"), "w") as metadata_file:
                json.dump(metadata, metadata_file)
                
            #print(str(counter) + " " + file.name + " processed!")
            
        except Exception as error:
            print(error)
        finally:
            counter += 1
            
# let gc!
gc.collect()

# Warning: Run next cell just once!

In [ ]:
# don't run this cell multiple times
metadata_files = glob.glob(os.path.join(metadata_save_dir, "*.json"))

In [ ]:
counter = 0
graph = dict()
for metadata_file in metadata_files:
    if counter < 100:
        print(metadata_file)
        counter += 1
    else:
        continue
    with io.open(metadata_file) as metadata:
        metadata = json.load(metadata)
        if metadata['From'] in graph:
            graph[metadata['From']].extend(metadata['To'])
        else:
            graph[metadata['From']] = (metadata['To'])

In [ ]:
with io.open("./datasets/processed/enron_graph.tsv", "w") as enron_graph:
    enron_graph.write("source\ttarget\tweight\n")
    for k, v in graph.items():
        source = k
        targets = [(target, v.count(target)) for target in v]
        for pair in targets:
            target, weight = pair
            enron_graph.write(source+"\t"+target+"\t"+str(weight)+"\n")

In [ ]:
enron_graph = pandas.read_csv("./datasets/processed/enron_graph.tsv", sep="\t")

In [ ]:
enron_graph.head(10)

In [ ]:
groupby_people = enron_graph.groupby(by=["source", "target"])

# unique emails(a.k.a graph nodes)
groupby_people.count().reset_index()['source']

In [ ]:
 metadata['To']